<a href="https://colab.research.google.com/github/sriprad/DeepLearning/blob/master/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-878.91626, shape=(), dtype=float32)


In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [ ]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
# df = pd.read_csv("C:/Users/P.SridharanSrinivas/Desktop/sai_pycaret/New_try/keras/NLP/train.csv")

In [ ]:
df1 = df.copy()


In [ ]:
df1 = df1.dropna()

In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
X = df1.drop('label',axis=1)
print(df)
# df1 = df.copy(deep=True)

          id  ... label
0          0  ...     1
1          1  ...     0
2          2  ...     1
3          3  ...     1
4          4  ...     1
...      ...  ...   ...
20795  20795  ...     0
20796  20796  ...     0
20797  20797  ...     0
20798  20798  ...     1
20799  20799  ...     1

[20800 rows x 5 columns]


In [ ]:
y=df1['label']

In [ ]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [ ]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
#voca size
voc_size =5000

In [ ]:
messages=X.copy()

In [ ]:
messages.reset_index(inplace=True)
# messages.dropna()

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [ ]:
onehot_repr =[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4973, 140, 4170, 2748, 114, 1736, 1279, 3596, 1078, 420],
 [3549, 1359, 554, 3219, 914, 20, 3651],
 [2999, 2917, 352, 2489],
 [4028, 4133, 2662, 1793, 644, 4341],
 [508, 914, 2097, 2342, 4248, 2917, 914, 3561, 2707, 3477],
 [1436,
  1621,
  4301,
  1546,
  2963,
  2561,
  2201,
  1520,
  4791,
  4394,
  176,
  4996,
  3032,
  4018,
  3651],
 [4167, 4524, 1844, 4677, 252, 3669, 4769, 3909, 3461, 2688, 1356],
 [2882, 3425, 3689, 2216, 2930, 1877, 2561, 1570, 3461, 2688, 1356],
 [3661, 628, 2962, 3960, 279, 492, 870, 4305, 2561, 2400],
 [2132, 2995, 2982, 3602, 3805, 4878, 581, 3785],
 [3837, 4729, 434, 4692, 1062, 4032, 4036, 2241, 3130, 4586, 2976],
 [1793, 492, 114, 492, 2561, 2930],
 [3450, 2834, 3370, 4200, 337, 191, 702, 1935, 2087],
 [1300, 4402, 4538, 1155, 534, 2036, 4736, 3461, 2688, 1356],
 [1746, 453, 3062, 1489, 1765, 3461, 2688, 1356],
 [2805, 864, 542, 142, 814, 4663, 1819, 2455, 1978, 1798],
 [4153, 642, 1359],
 [4974, 4679, 4172, 1667, 2561, 1224, 2664, 3651],
 [3345, 2

In [ ]:
#embedding representation

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3596 1078  420]
 [   0    0    0 ...  914   20 3651]
 [   0    0    0 ... 2917  352 2489]
 ...
 [   0    0    0 ... 3461 2688 1356]
 [   0    0    0 ...  407 3786  642]
 [   0    0    0 ...  893 2460 4516]]


In [ ]:

embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4973,
        140, 4170, 2748,  114, 1736, 1279, 3596, 1078,  420], dtype=int32)

In [ ]:
#creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final =np.array(embedded_docs)
y_final =np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
y_test.shape

(6035,)

In [ ]:
#model training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0022 - accuracy: 0.9993 - val_loss: 0.6743 - val_accuracy: 0.9084
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.4493 - val_accuracy: 0.9054
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0112 - accuracy: 0.9962 - val_loss: 0.5314 - val_accuracy: 0.9087
Epoch 4/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.5687 - val_accuracy: 0.9064
Epoch 5/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0024 - accuracy: 0.9997 - val_loss: 0.7187 - val_accuracy: 0.9074
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 3.2882e-04 - accuracy: 1.0000 - val_loss: 0.7823 - val_accuracy: 0.9082
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 1.3404e-04 - accuracy: 1.0000 - val_loss: 0.8089 - val_accuracy

In [ ]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3116,  303],
       [ 261, 2355]])

In [ ]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9065451532725767